In [1]:
import time
import html
import numbers
import functools
from collections.abc import MutableSequence

A decorator is a callable that takes another function as argument (the decorated function). 2 The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object. In other words, assuming an existing decorator named decorate, this code:

```
@decorate
def target():
    print('running target()')
```

Has the same effect as writing this:

```
def target():
    print('running target()')


target = decorate(target)
```

In [2]:
def deco(func):
    def inner():
        print("Running inner()")
        func()
    return inner

In [3]:
@deco
def target():
    print("Running target()")

In [4]:
target()

Running inner()
Running target()


In [5]:
target

<function __main__.deco.<locals>.inner()>

## When Python Executes Decorators
***A key feature of decorators is that they run right after the decorated function is defined. That is usually at import time (i.e., when a module is loaded by Python)***.

In [6]:
!python registration.py

Running register(<function f1 at 0x000001E9D7477AF0>)
Running register(<function f2 at 0x000001E9D7477B80>)
running main()
registry -> [<function f1 at 0x000001E9D7477AF0>, <function f2 at 0x000001E9D7477B80>]
running f1()
running f2()
running f3()


Note that register runs (twice) before any other function in the module. When reg ister is called, it receives as an argument the function object being decorated—for example, <function f1 at 0x100631bf8>. After the module is loaded, the registry holds references to the two decorated functions: f1 and f2. These functions, as well as f3, are only executed when explicitly called
by main.

In [7]:
import registration

Running register(<function f1 at 0x0000021469410940>)
Running register(<function f2 at 0x00000214694103A0>)


In [8]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

The main point of above Example is to emphasize that function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked. This highlights the difference between what Pythonistas call import time and runtime.

## Decorator-Enhanced Strategy Pattern

In [9]:
promos = []

In [10]:
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

In [11]:
@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [12]:
@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [13]:
@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [14]:
def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

## Variable Scope Rules

In [15]:
def f1(a):
    print(a)
    print(b)

In [16]:
try:
    f1(2)
except NameError:
    print("name 'b' is not defined")

2
name 'b' is not defined


In [17]:
b = 6
f1(3)

3
6


In [18]:
def f2(a):
    print(a)
    print(b)
    b = 9

In [19]:
try:
    f2(3)
except UnboundLocalError:
    print("UnboundLocalError: local variable 'b' referenced before assignment")

3
UnboundLocalError: local variable 'b' referenced before assignment


When Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch b from the local environment. Later, when the call f2(3)
is made, the body of f2 fetches and prints the value of the local variable a, but when trying to fetch the value of local variable b it discovers that b is unbound.

This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local. This is much better than the behavior of JavaScript, which does not require variable declarations either, but if you do forget to declare that a variable is local (with var), you may clobber a global variable without knowing.

## Closures
A closure is a function with an extended scope that encompasses nonglobal
variables referenced in the body of the function but not defined there. It does not matter
whether the function is anonymous or not; what matters is that it can access nonglobal
variables that are defined outside of its body.

### A class to calculate a running average

In [20]:
class Averager:
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [21]:
avg = Averager()
avg(10)

10.0

In [22]:
avg(11)

10.5

In [23]:
avg(12)

11.0

### A higher-order function to calculate a running average

In [24]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [25]:
avg = make_averager()
avg(10)

10.0

In [26]:
avg(11)

10.5

It’s obvious where the avg of the `Averager` class keeps the history: the `self.series` instance attribute. But where does the avg function in the second example find the series?

Note that series is a local variable of make_averager because the initialization `series = []` happens in the body of that function. But when `avg(10)` is called, `make_averager` has already returned, and its local scope is long gone.

Within averager, series is a free variable. This is a technical term meaning a variable that is not bound in the local scope.

Inspecting the returned averager object shows how Python keeps the names of `local` and `free variables` in the `__code__` attribute that represents the compiled body of the function.

In [27]:
avg.__code__.co_varnames

('new_value', 'total')

In [28]:
avg.__code__.co_freevars

('series',)

The binding for series is kept in the `__closure__` attribute of the returned function `avg`. Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_free` vars. These items are cells, and they have an attribute called cell_contents where
the actual value can be found.

In [29]:
avg.__closure__

(<cell at 0x0000021469431C10: list object at 0x0000021469405500>,)

In [30]:
avg.__closure__[0].cell_contents

[10, 11]

**To summarize:** a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

## The nonlocal Declaration
Our previous implementation of make_averager was not efficient. In Example above, we stored all the values in the historical series and computed their sum every time averager was called. A better implementation would just store the total and the number of items
so far, and compute the mean from these two numbers.

### A broken higher-order function to calculate a running average without keeping all history

In [31]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [32]:
avg = make_averager()
try:
    avg(10)
except UnboundLocalError:
    print("UnboundLocalError: local variable 'count' referenced before assignment")

UnboundLocalError: local variable 'count' referenced before assignment


The problem is that the statement `count += 1` actually means the same as `count = count + 1`, when count is a number or any immutable type. So we are actually assigning to count in the body of averager, and that makes it a local variable.

To work around this, the `nonlocal` declaration was introduced in Python 3. It lets you flag a variable as a free variable even when it is assigned a new value within the function.

In [33]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [34]:
avg = make_averager()
avg(10)

10.0

In [35]:
avg(11)

10.5

In [36]:
avg(12)

11.0

## Implementing a Simple Decorator

In [37]:
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f"[{elapsed:.8f}] {name}(){arg_str} -> {result}")
        return result
    return clocked

In [38]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [39]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [40]:
print('*' * 40, '\nCalling snooze(.123)')
snooze(.123)
print('*' * 40, '\nCalling factorial(10)')
print('10! =', factorial(10))

**************************************** 
Calling snooze(.123)
[0.12319720] snooze()0.123 -> None
**************************************** 
Calling factorial(10)
[0.00000090] factorial()1 -> 1
[0.00005530] factorial()2 -> 2
[0.00009050] factorial()3 -> 6
[0.00012490] factorial()4 -> 24
[0.00015900] factorial()5 -> 120
[0.00019560] factorial()6 -> 720
[0.00022800] factorial()7 -> 5040
[0.00025770] factorial()8 -> 40320
[0.00084820] factorial()9 -> 362880
[0.00104260] factorial()10 -> 3628800
10! = 3628800


In [41]:
factorial.__name__

'clocked'

So factorial now actually holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed.

The clock decorator implemented in Example 7-15 has a few shortcomings: it does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function.

### An improved clock decorator

In [42]:
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [43]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [44]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [45]:
print('*' * 40, '\nCalling snooze(.123)')
snooze(.123)
print('*' * 40, '\nCalling factorial(10)')
print('10! =', factorial(10))

**************************************** 
Calling snooze(.123)
[0.12792683s] snooze(0.123) -> None 
**************************************** 
Calling factorial(10)
[0.00000000s] factorial(1) -> 1 
[0.00099993s] factorial(2) -> 2 
[0.00099993s] factorial(3) -> 6 
[0.00099993s] factorial(4) -> 24 
[0.00099993s] factorial(5) -> 120 
[0.00099993s] factorial(6) -> 720 
[0.00299811s] factorial(7) -> 5040 
[0.00299811s] factorial(8) -> 40320 
[0.00299811s] factorial(9) -> 362880 
[0.00299811s] factorial(10) -> 3628800 
10! = 3628800


In [46]:
factorial.__name__

'factorial'

## Memoization with functools.lru_cache

In [47]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [48]:
print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(5) -> 5 
[0.00099945s] fibonacci(6) -> 8 
8


The waste is obvious: fibonacci(1) is called eight times, fibonacci(2) five times, etc. But if we just add two lines to use lru_cache, performance is much improved.

In [49]:
@functools.lru_cache() #
@clock #
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [50]:
print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00299954s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00299954s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(5) -> 5 
[0.00399780s] fibonacci(6) -> 8 
8


## Generic Functions with Single Dispatch

In [51]:
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [52]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [53]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [54]:
htmlize(42)

'<pre>42</pre>'

In [55]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


Because we don’t have method or function overloading in Python, we can’t create variations of htmlize with different signatures for each data type we want to handle differently.

A common solution in Python would be to turn htmlize into a dispatch function, with a chain of if/elif/elif calling specialized functions like htmlize_str, htmlize_int, etc.

In [56]:
@functools.singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [57]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

In [58]:
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

In [59]:
@htmlize.register(tuple)
@htmlize.register(MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

## Parameterized Decorators
### To accept parameters, the new register decorator must be called as a function

In [60]:
import registration_param

running register(active=False)->decorate(<function f1 at 0x000002146945BAF0>)
running register(active=True)->decorate(<function f2 at 0x000002146945BD30>)


In [61]:
registration_param.registry

{<function registration_param.f2()>}

In [62]:
from registration_param import *

In [63]:
registry

{<function registration_param.f2()>}

In [64]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x000002146945BA60>)


<function registration_param.f3()>

In [65]:
registry

{<function registration_param.f2()>, <function registration_param.f3()>}

In [66]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x000002146945BD30>)


<function registration_param.f2()>

In [67]:
registry

{<function registration_param.f3()>}

### The parameterized clock decorator

In [68]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

In [69]:
def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [70]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

In [71]:
for i in range(3):
    snooze(.123)

[0.14791679s] snooze(0.123) -> None
[0.12808466s] snooze(0.123) -> None
[0.12435579s] snooze(0.123) -> None


In [72]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

In [73]:
for i in range(3):
    snooze(.123)

snooze: 0.13182997703552246s
snooze: 0.12357354164123535s
snooze: 0.12360095977783203s


In [74]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

In [75]:
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.136s
snooze(0.123) dt=0.125s
snooze(0.123) dt=0.123s


Parameterized decorators usually replace the decorated function, and their construction requires yet another level of nesting.